In [1]:
socketDF = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

In [2]:
socketDF.isStreaming
tmpSchema = socketDF.schema
tmpSchema 

In [3]:
csvDF = spark.readStream\
.option("sep", " ")\
.schema(tmpSchema )\
.csv("/FileStore/tables/")

In [4]:
from pyspark.sql.functions import explode, split
words = csvDF.select(explode(split(csvDF.value, " ")).alias("word"))

In [5]:
wordCounts = words.groupBy("word").count()
print wordCounts

In [6]:
#wordCounts.writeStream\
#.outputMode("complete")\
#.format("console")\
#.start()
runningStream = wordCounts.writeStream\
.format("memory")\
.queryName("myResult")\
.outputMode("complete")\
.start()

In [7]:
dfRes = spark.sql("select * from myResult")

In [8]:
dfRes.write.format("com.databricks.spark.csv").save("/FileStore/tables/result.csv")